NoSQL datapipeline
Apache Airflow - Orchestrador e Agendador
OpenWeatherAPI - api de tempo
Python (extrair api)
Pyspark - Transformar, validar e carregar
MongoDB
Criaçao de Dashboard interativo

In [3]:
%cd /content/drive/MyDrive/github_projects/openweather-pyspark-nosql

/content/drive/MyDrive/github_projects/openweather-pyspark-nosql


In [33]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (74.0 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 126102 files and directories currently instal

In [12]:
!tree -L 2

.
├── data
│   ├── processed
│   └── raw
├── docker
│   ├── airflow
│   ├── notebook
│   └── requirements.txt
├── info.txt
├── mnt
│   ├── airflow
│   ├── notebook
│   └── python
├── README.md
└── services

11 directories, 3 files


In [36]:
!pip install pyspark findspark

In [7]:
!rm -rf "/content/drive/MyDrive/github_projects/openweather-pyspark-nosql/data/processed"

In [4]:
from mnt.python.coletor_weather import ColetorWeather

# Lista de cidades (ex: capitais)
cidades = [
    'Rio Branco,BR',        # Acre (AC)
    'Maceió,BR',               # Alagoas (AL)
    'Macapá,BR',               # Amapá (AP)
    'Manaus,BR',               # Amazonas (AM)
    'Salvador,BR',             # Bahia (BA)
    'Fortaleza,BR',            # Ceará (CE)
    'Brasília,BR',             # Distrito Federal (DF)
    'Vitória,BR',              # Espírito Santo (ES)
    'Goiânia,BR',              # Goiás (GO)
    'São Luís,BR',             # Maranhão (MA)
    'Cuiabá,BR',               # Mato Grosso (MT)
    'Campo Grande,BR',         # Mato Grosso do Sul (MS)
    'Belo Horizonte,BR',       # Minas Gerais (MG)
    'Belém,BR',                # Pará (PA)
    'João Pessoa,BR',          # Paraíba (PB)
    'Curitiba,BR',             # Paraná (PR)
    'Recife,BR',               # Pernambuco (PE)
    'Teresina,BR',             # Piauí (PI)
    'Rio de Janeiro,BR',       # Rio de Janeiro (RJ)
    'Natal,BR',                # Rio Grande do Norte (RN)
    'Porto Alegre,BR',         # Rio Grande do Sul (RS)
    'Porto Velho,BR',          # Rondônia (RO)
    'Boa Vista,BR',            # Roraima (RR)
    'Florianópolis,BR',        # Santa Catarina (SC)
    'São Paulo,BR',            # São Paulo (SP)
    'Aracaju,BR',              # Sergipe (SE)
    'Palmas,BR'                # Tocantins (TO)
]

# Instancia e coleta
extract = ColetorWeather(cidades)
dados = extract.coletar()

# Salvar dados
extract.salvar_jsons(dados)

Coletado: Rio Branco,BR
Coletado: Maceió,BR
Coletado: Macapá,BR
Coletado: Manaus,BR
Coletado: Salvador,BR
Coletado: Fortaleza,BR
Coletado: Brasília,BR
Coletado: Vitória,BR
Coletado: Goiânia,BR
Coletado: São Luís,BR
Coletado: Cuiabá,BR
Coletado: Campo Grande,BR
Coletado: Belo Horizonte,BR
Coletado: Belém,BR
Coletado: João Pessoa,BR
Coletado: Curitiba,BR
Coletado: Recife,BR
Coletado: Teresina,BR
Coletado: Rio de Janeiro,BR
Coletado: Natal,BR
Coletado: Porto Alegre,BR
Coletado: Porto Velho,BR
Coletado: Boa Vista,BR
Coletado: Florianópolis,BR
Coletado: São Paulo,BR
Coletado: Aracaju,BR
Coletado: Palmas,BR
Total de cidades coletadas: 27
Arquivos salvos em: /content/drive/MyDrive/github_projects/openweather-pyspark-nosql/data/raw/ano=2025/mes=05/dia=24/hora=15


In [10]:
import json
import pandas as pd

with open('/content/drive/MyDrive/github_projects/openweather-pyspark-nosql/data/raw/ano=2025/mes=05/dia=24/hora=15/clima_Rio_de_Janeiro_2025-05-24_15-48-13.json', 'r', encoding='utf-8') as f:
    dados = json.load(f)

df = pd.json_normalize(dados)

display(df)

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.sea_level,main.grnd_level,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1748101395,-10800,3451190,Rio de Janeiro,200,-43.2075,-22.9028,...,1023,1024,1.54,350,75,2,2098643,BR,1748078576,1748117806


In [9]:
from mnt.python.transformador_weather import TransformadorClima

#spark = SparkSession.builder.appName("TransformadorClima").getOrCreate()
transformador = TransformadorClima()

df_raw = transformador.ler_json("data/raw/ano=2025/mes=05/dia=24/hora=15")
df_transformado = transformador.transformar(df_raw)
transformador.salvar_parquet(df_transformado)


Dados salvos em: data/processed/ano=2025/mes=05/dia=24/hora=15


In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LeituraParquet").getOrCreate()

# Caminho onde estão os arquivos parquet
caminho = "data/processed/ano=2025/mes=05/dia=24/hora=15"

df = spark.read.parquet(caminho)
df.show()

+--------------+----------+-----------+-------+----------------+-----------------+-------------------+-----------------+--------------------+
|        cidade| timestamp|temperatura|umidade|velocidade_vento|  descricao_clima|          data_hora|faixa_temperatura|         data_coleta|
+--------------+----------+-----------+-------+----------------+-----------------+-------------------+-----------------+--------------------+
|      Salvador|1748101691|      26.98|     74|            5.63|céu pouco nublado|2025-05-24 15:48:11|        Agradável|2025-05-24 15:49:...|
|Rio de Janeiro|1748101395|      22.05|     80|            1.54| nuvens quebradas|2025-05-24 15:43:15|        Agradável|2025-05-24 15:49:...|
|Belo Horizonte|1748101578|      24.46|     60|             3.6| nuvens dispersas|2025-05-24 15:46:18|        Agradável|2025-05-24 15:49:...|
|   João Pessoa|1748101692|      29.13|     74|            4.63| nuvens quebradas|2025-05-24 15:48:12|        Agradável|2025-05-24 15:49:...|
|    R

In [ ]:
from mnt.python.load_weather_mongo import LoadMongo

mongo_loader = LoadMongo()

df = mongo_loader.ler_parquet("data/processed/ano=2025/mes=05/dia=24/hora=15")
mongo_loader.escrever_mongodb(df)


TESTE

In [14]:
#!pip install dnspython
!pip install pymongo[srv] dnspython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.1 MB/s eta 0:00:00


In [11]:
!spark-submit --version


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.27
Branch HEAD
Compiled by user heartsavior on 2024-02-15T11:24:58Z
Revision fd86f85e181fc2dc0f50a096855acf83a6cc5d9c
Url https://github.com/apache/spark
Type --help for more information.


In [16]:

!mkdir -p /content/jars

# Conector Spark-MongoDB
!wget https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/10.2.0/mongo-spark-connector_2.12-10.2.0.jar -P /content/jars

# Driver Java do MongoDB
!wget https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-sync/4.10.2/mongodb-driver-sync-4.10.2.jar -P /content/jars


--2025-05-24 16:55:12--  https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/10.2.0/mongo-spark-connector_2.12-10.2.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172328 (168K) [application/java-archive]
Saving to: ‘/content/jars/mongo-spark-connector_2.12-10.2.0.jar’

mongo-spark-connect 100%[===================>] 168.29K  --.-KB/s    in 0.03s   

2025-05-24 16:55:12 (6.38 MB/s) - ‘/content/jars/mongo-spark-connector_2.12-10.2.0.jar’ saved [172328/172328]

--2025-05-24 16:55:12--  https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-sync/4.10.2/mongodb-driver-sync-4.10.2.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.


In [3]:
%cd /content/drive/MyDrive/github_projects/openweather-pyspark-nosql

/content/drive/MyDrive/github_projects/openweather-pyspark-nosql


In [24]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

# Carrega variáveis do .env
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")

spark = SparkSession.builder \
    .appName("MongoSparkConnector") \
    .config("spark.jars", "mongo-java-driver-3.12.10.jar,mongo-spark-connector_2.12-10.2.0.jar") \
    .config("spark.mongodb.write.connection.uri", MONGO_URI) \
    .getOrCreate()

print(spark)

In [20]:
spark._jvm.com.mongodb.ConnectionString

In [26]:
df = spark.read.parquet("/content/drive/MyDrive/github_projects/openweather-pyspark-nosql/data/processed/ano=2025/mes=05/dia=24/hora=15")
df.show()


+--------------+----------+-----------+-------+----------------+-----------------+-------------------+-----------------+--------------------+
|        cidade| timestamp|temperatura|umidade|velocidade_vento|  descricao_clima|          data_hora|faixa_temperatura|         data_coleta|
+--------------+----------+-----------+-------+----------------+-----------------+-------------------+-----------------+--------------------+
|      Salvador|1748101691|      26.98|     74|            5.63|céu pouco nublado|2025-05-24 15:48:11|        Agradável|2025-05-24 15:49:...|
|Rio de Janeiro|1748101395|      22.05|     80|            1.54| nuvens quebradas|2025-05-24 15:43:15|        Agradável|2025-05-24 15:49:...|
|Belo Horizonte|1748101578|      24.46|     60|             3.6| nuvens dispersas|2025-05-24 15:46:18|        Agradável|2025-05-24 15:49:...|
|   João Pessoa|1748101692|      29.13|     74|            4.63| nuvens quebradas|2025-05-24 15:48:12|        Agradável|2025-05-24 15:49:...|
|    R

In [ ]:
df.write \
    .format("mongodb") \
    .mode("append") \
    .option("database", "clima") \
    .option("collection", "dados") \
    .save()


In [10]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure


try:
    client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
    client.server_info()
    print("Conexão com MongoDB estabelecida com sucesso!")
except ConnectionFailure as e:
    print("Falha na conexão com MongoDB:", e)


[✓] Conexão com MongoDB estabelecida com sucesso!
